In [1]:
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource
import pickle
import numpy as np
from BadLoanVerifier import BankLoanVerifier

In [2]:
app = Flask(__name__)
api = Api(app)



clf_path = './BadLoanVerifier.pkl'
with open(clf_path, 'rb') as f:
    model = pickle.load(f)
    
parser = reqparse.RequestParser()
parser.add_argument('loan_amnt')
parser.add_argument('term')
parser.add_argument('emp_length')
parser.add_argument('home_ownership')
parser.add_argument('annual_inc')
parser.add_argument('purpose')
parser.add_argument('addr_state')
parser.add_argument('dti')
parser.add_argument('delinq_2yrs')
parser.add_argument('revol_util')
parser.add_argument('total_acc')
parser.add_argument('longest_credit_length')
parser.add_argument('verification_status')



In [3]:
class PredictBadLoan(Resource):
    def get(self):
        # use parser and find the user's query
        args = parser.parse_args()
        loan_amnt = args['loan_amnt']       
        term= args['term']
        emp_length= args['emp_length']
        home_ownership= args['home_ownership']
        annual_inc= args['annual_inc']
        purpose= args['purpose']
        addr_state= args['addr_state']
        dti= args['dti']
        delinq_2yrs= args['delinq_2yrs']
        revol_util= args['revol_util']
        total_acc= args['total_acc']
        longest_credit_length= args['longest_credit_length']
        verification_status= args['verification_status']
        
        
        sample=pd.DataFrame(dict({'loan_amnt':[loan_amnt],'term':[term],'emp_length':[emp_length],
                                  'home_ownership':[home_ownership],'annual_inc':[annual_inc],'purpose':[purpose],
                                  'addr_state':[addr_state],'dti':[dti],'delinq_2yrs':[delinq_2yrs],'revol_util':[revol_util],
                                  'total_acc':[total_acc],'longest_credit_length':[longest_credit_length],'verification_status':[verification_status]}))
        prediction = model.predict(sample)[0]
        prediction_proba=model.predict_proba(sample,prediction)
        if prediction == 0:
            pred_text = 'Verified'
        else:
            pred_text = 'Not Verified'
            
        # round the predict proba value and set to new variable
        confidence = round(prediction_proba, 3)        # create JSON object
        output = {'prediction': pred_text, 'confidence': confidence}
        
        return output
    
    

In [4]:
api.add_resource(PredictBadLoan, '/')


if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb